In [3]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
        print ("step %d, training accuracy %g"%(i, train_accuracy))
print ("test accuracy",accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
print (mnist.test.labels)
print (mnist.test.images)

/home/fly/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.22
step 100, training accuracy 0.94
step 200, training accuracy 0.96
step 300, training accuracy 0.92
step 400, training accuracy 0.94
step 500, training accuracy 0.94
step 600, training accuracy 0.88
step 700, training accuracy 0.98
step 800, training accuracy 0.92
step 900, training accuracy 0.92
test accuracy 0.9109
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')


In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


In [ ]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


In [ ]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
saver = tf.train.Saver()
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs, y_: batch_ys, keep_prob: 1.0})
        print ("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})

saver.save(sess, './save_model_data_conv/model.ckpt')   
test_accuracy=accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
print("test accuracy",test_accuracy)

In [ ]:
for i in range(0, len(mnist.test.images)):
  result = sess.run(correct_prediction, feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])})
  if not result:
    print('预测的值是：',sess.run(y, feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])}))
    print('实际的值是：',sess.run(y_,feed_dict={x: np.array([mnist.test.images[i]]), y_: np.array([mnist.test.labels[i]])}))
    one_pic_arr = np.reshape(mnist.test.images[i], (28, 28))
    pic_matrix = np.matrix(one_pic_arr, dtype="float")
    plt.imshow(pic_matrix)
    pylab.show()
    break
 
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                     y_: mnist.test.labels}))

In [ ]:
import os
import struct
import numpy as np
def checkIsBmp(file):
    with open(file,'rb') as f:
        head = struct.unpack('ccIIIIIIHH',f.read(32))
        if head[0] == b'B' and head[1] == b'M':
            return True,head[2],head[9]
        else:
            return False ,0,0
def test_my_data():
    with tf.Session() as sess:
        model_data = tf.train.latest_checkpoint('./save_model_data_conv/')
        saver.restore(sess,model_data)
        print('Start recognizing my image:')
        my_data_path = './my_test_images/'
        print(os.listdir(my_data_path))
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        for image_name in os.listdir(my_data_path):
            image_path = os.path.join(my_data_path, image_name)
            ret, file_size, bitCnt_per_pix = checkIsBmp(image_path)
            if ret == True:
                with open(image_path, 'rb') as f:
                    formate = '%dB' % file_size
                    data_bytes = struct.unpack(formate, f.read(file_size)) #按unsigned char 读取文件
                    image_np = np.zeros((1, 784))
                    step = bitCnt_per_pix / 8
                    start_pos = 54 
                    if bitCnt_per_pix == 8: 
                        start_pos = start_pos + 1024 
                    for i in range(784):
                        pos = start_pos + i * step
                        if bitCnt_per_pix == 8:
                            pix_value = data_bytes[pos]
                        elif bitCnt_per_pix == 24:
                            pix_value = data_bytes[pos] * 0.299 + data_bytes[pos+1] * 0.587 + data_bytes[pos+2] * 0.114
                        image_np[0][i] = pix_value / 255.0 # [0..255] --> [0.0... 1.0]
                    max_idx, out = sess.run([tf.argmax(y, 1), y], feed_dict = {x: image_np, keep_prob: 1})
                    print('####The image name: %s, predict number is: %d' % (image_name, max_idx))

test_my_data()
